<a href="https://colab.research.google.com/github/TovarischComrad/Tehnopark/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip archive6.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: dataset/dataset/ceratitis-capitata/ceratitis-capitata_92_1364760f.jpg  
  inflating: dataset/dataset/ceratitis-capitata/ceratitis-capitata_93_df914c32.jpg  
  inflating: dataset/dataset/ceratitis-capitata/ceratitis-capitata_94_fd5647fe.jpg  
  inflating: dataset/dataset/ceratitis-capitata/ceratitis-capitata_96_b1b70c45.jpg  
  inflating: dataset/dataset/ceratitis-capitata/ceratitis-capitata_97_b7c264e5.jpg  
  inflating: dataset/dataset/ceratotherium-simum/ceratotherium-simum_18_4fcf8ad3.jpg  
  inflating: dataset/dataset/ceratotherium-simum/ceratotherium-simum_18_7565c9eb.jpg  
  inflating: dataset/dataset/ceratotherium-simum/ceratotherium-simum_19_8b607f2d.jpg  
  inflating: dataset/dataset/ceratotherium-simum/ceratotherium-simum_19_ced6ec21.jpg  
  inflating: dataset/dataset/ceratotherium-simum/ceratotherium-simum_27_44d0cb97.jpg  
  inflating: dataset/dataset/ceratotherium-simum/ceratotherium-simum_40_

In [7]:
import json

N = 15

file = open('translation.json')
db = json.load(file)
keys = list(db.keys())
names = keys[:N]
names

['equus-quagga',
 'panthera-leo',
 'panthera-onca',
 'ursus-arctos-horribilis',
 'megaptera-novaeangliae',
 'panthera-tigris',
 'haliaeetus-leucocephalus',
 'tyrannosaurus-rex',
 'diplodocus',
 'panthera-pardus',
 'homo-sapiens',
 'phoebetria-fusca',
 'bos-taurus',
 'gallus-gallus-domesticus',
 'ovis-aries']

In [8]:
cl = [[i, names[i]] for i in range(N)]
cl

[[0, 'equus-quagga'],
 [1, 'panthera-leo'],
 [2, 'panthera-onca'],
 [3, 'ursus-arctos-horribilis'],
 [4, 'megaptera-novaeangliae'],
 [5, 'panthera-tigris'],
 [6, 'haliaeetus-leucocephalus'],
 [7, 'tyrannosaurus-rex'],
 [8, 'diplodocus'],
 [9, 'panthera-pardus'],
 [10, 'homo-sapiens'],
 [11, 'phoebetria-fusca'],
 [12, 'bos-taurus'],
 [13, 'gallus-gallus-domesticus'],
 [14, 'ovis-aries']]

In [9]:
import glob
from PIL import Image as im
import numpy as np

In [11]:
data = []
for i in range(N):
    file = f'/content/dataset/dataset/{cl[i][1]}/*.jpg'
    for filename in glob.glob(file):
        img = im.open(filename)
        arr = np.asarray(img)
        data.append([arr, cl[i][0]])

In [12]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.3)
data_train, data_val = train_test_split(data_train, test_size=0.2)

X_train = np.array([el[0] for el in data_train])
y_train = np.array([el[1] for el in data_train])

X_test = np.array([el[0] for el in data_test])
y_test = np.array([el[1] for el in data_test])

X_val = np.array([el[0] for el in data_val])
y_val = np.array([el[1] for el in data_val])

In [13]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

In [14]:
vgg_16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_16.trainable = False
vgg_16.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [16]:
model = Sequential()
model.add(vgg_16)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(N))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         3,855 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 15)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,141,327 (80.65 MB)

 Trainable params: 6,426,639 (24.52 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [15]:
batch_size = N
N_train = len(data_train)
N_test = len(data_test)
N_val = len(data_val)

In [18]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=batch_size,
    validation_data=(X_val, y_val)
)

Epoch 1/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 265s 13s/step - accuracy: 0.1994 - loss: 57.2727 - val_accuracy: 0.7027 - val_loss: 8.4399
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 257s 13s/step - accuracy: 0.7480 - loss: 11.1087 - val_accuracy: 0.6216 - val_loss: 9.6587
Epoch 3/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 261s 13s/step - accuracy: 0.8505 - loss: 4.4200 - val_accuracy: 0.7568 - val_loss: 8.0958
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 224s 11s/step - accuracy: 0.8786 - loss: 2.7078 - val_accuracy: 0.7432 - val_loss: 8.1372
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 299s 13s/step - accuracy: 0.9003 - loss: 1.4169 - val_accuracy: 0.7568 - val_loss: 6.9179
